# Reventar a las casas de apuestas

### Imports and options

In [3]:
import sqlite3
from bs4 import BeautifulSoup

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

options = webdriver.ChromeOptions()
options.add_argument('ignore-certificate-errors')
#options.add_argument('--headless')
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
#options.add_experimental_option('excludeSwitches', ['enable-logging'])

driver = webdriver.Chrome(options=options)

In [9]:
class Bet():
    def __init__(self, teams, bets):
        self.teams = teams
        self.bets = bets

    def __repr__(self):
        return f"{self.teams[0]} ({self.bets[0]}) x {self.teams[1]} ({self.bets[1]})"

In [7]:
class Retabet():
    def __init__(self):
        self.link = "https://apuestas.retabet.es"
        self.sports = {"tennis" : "tenis-m8"}

    def link_sport(self, sport):
        return f"{self.link}/deportes/{self.sports[sport]}"

    def sport_bets(self, sport):
        driver.get(self.link_sport(sport))
        response = driver.page_source
        #f = open("out", "w+")
        #print(html.encode('utf-8'), file=f)
        soup = BeautifulSoup(response, "html.parser")
        table = soup.find('ul', attrs={'class':'module__events-cont jbgroup'}) #TODO findAll
        ret_bets = []
        for row in table:
            row = row.find('a', attrs={'class':'jlink'})
            teams = row.find('div', attrs={'class':'module__event-info'})
            teams = teams.find('ul', attrs={'class':'module__event-players'})
            teams = list(map(lambda elem: elem.text, teams.findAll('li')))

            bets = row.find('div', attrs={'class':'list-events__element__bets'})
            bets = row.find('div', attrs={'class':'module__bets-list jbet'})
            bets = row.findAll('li', attrs={'class':'jo module__bets-li'})
            bets_list = []
            for bet, _ in zip(bets, range(2)):
                bet = bet.find('span', attrs={'class':'jpr module__bets-cuota'})
                bets_list.append(bet.text)

            bet = Bet(teams, bets_list)
            ret_bets.append(bet)
        return ret_bets

In [10]:
reta = Retabet()
reta.sport_bets("tennis")

F. PASSARO (2,05) x D. POPKO (1,70)

In [20]:
driver.get("https://sports.bwin.es/es/sports")

In [2]:
driver.get("https://m.apuestas.codere.es/deportes/index.htm#/HomePage")

In [4]:
driver.get("https://sports.williamhill.es/betting/es-es")

In [5]:
driver.get("https://betway.es/es/sports")

In [ ]:
<span class="teamNameFirstPart teamNameHomeTextFirstPart">Elias Ymer</span>